In [1]:
import numpy as np
import keras
import os
import os.path
import cv2 as cv
import keras.layers as l
from keras.models import Sequential

Using TensorFlow backend.


In [2]:
path = '/mnt/data/deepfake/datasets/faces/'
dataset_1 = '/mnt/data/deepfake/datasets/faces/Actor_01/'
dataset_2 = '/mnt/data/deepfake/datasets/faces/Actor_02/'
DIM = 100

# Dataset 1

In [3]:
images_1 = os.listdir(path + '/Actor_01')
n_dataset_1 = len(images_1)
n_train_1 = int (n_dataset_1 * 0.9)
n_test_1 = n_dataset_1 - n_train_1

partition_1 = {'train':[], 'test':[]}
labels_1 = {}

for i in range (n_train_1):
    name = images_1[i]
    partition_1['train'].append(name)
    labels_1[name] = i
for i in range (n_train_1, n_dataset_1):
    name = images_1[i]
    partition_1['test'].append(name)
    labels_1[name] = i - n_train_1
    

# Dataset 2

In [4]:
images_2 = os.listdir(path + '/Actor_02')
n_dataset_2 = len(images_2)
n_train_2 = int (n_dataset_2 * 0.9)
n_test_2 = n_dataset_2 - n_train_2

partition_2 = {'train':[], 'test':[]}
labels_2 = {}

for i in range (n_train_2):
    name = images_2[i]
    partition_2['train'].append(name)
    labels_2[name] = i
for i in range (n_train_2, n_dataset_2):
    name = images_2[i]
    partition_2['test'].append(name)
    labels_2[name] = i - n_train_2


In [5]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(100,100), n_channels=1,
                 n_classes=10, shuffle=True, sample = 1):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.sample = sample
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        filename = 'Actor_0' + str(self.sample) + '/'
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp, filename)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp, filename):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        X.shape
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            
            img = (cv.resize(cv.cvtColor(cv.imread(path + filename + ID), cv.COLOR_BGR2GRAY), (DIM,DIM))) / 255.
            X[i,] = img.reshape((DIM, DIM,1))
            # Store class
            y[i] = self.labels[ID]

        return X, X

In [6]:
params = {'dim' : (100,100),
          'batch_size' : 64,
          'n_classes' : 6,
          'n_channels' : 1,
          'shuffle' : True,
          'sample' : 1}

In [7]:
training_generator_1 = DataGenerator(partition_1['train'], labels_1, **params)
validation_generator_1 = DataGenerator(partition_1['test'], labels_1, **params)

In [8]:
training_generator_2 = DataGenerator(partition_2['train'], labels_2, **params)
validation_generator_2 = DataGenerator(partition_2['test'], labels_2, **params)

In [150]:
input_layer = l.Input(shape = (DIM * DIM,))
encoded = l.Dense(32, activation='relu')(input_layer)
decoded_a = l.Dense(DIM*DIM, activation = "sigmoid")(encoded)
decoded_b = l.Dense(DIM*DIM, activation = "sigmoid")(encoded)
autoencodeur_a = keras.Model(input_layer, decoded_a)
autoencodeur_b = keras.Model(input_layer, decoded_b)
encodeur = keras.Model(input_layer, encoded)


# create a placeholder for an encoded (32-dimensional) input
encoded_input = l.Input(shape=(32,))
# retrieve the last layer of the autoencoder model
decoder_a_layer = autoencodeur_a.layers[-1]
decoder_b_layer = autoencodeur_b.layers[-1]

# create the decoder model
decodeur_a = keras.Model(encoded_input, decoder_a_layer(encoded_input))
decodeur_b = keras.Model(encoded_input, decoder_b_layer(encoded_input))

autoencodeur_a.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencodeur_b.compile(optimizer='adadelta', loss='binary_crossentropy')

In [9]:

input_img = l.Input(shape=(100, 100, 1))  # adapt this if using `channels_first` image data format

x = l.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = l.MaxPooling2D((2, 2), padding='same')(x)
x = l.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = l.MaxPooling2D((2, 2), padding='same')(x)
x = l.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = l.MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

xa = l.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
xa = l.UpSampling2D((2, 2))(xa)
xa = l.Conv2D(8, (3, 3), activation='relu', padding='same')(xa)
xa = l.UpSampling2D((2, 2))(xa)
xa = l.Conv2D(16, (3, 3), activation='relu')(xa)
xa = l.UpSampling2D((2, 2))(xa)
decoded_a = l.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(xa)

xb = l.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
xb = l.UpSampling2D((2, 2))(xb)
xb = l.Conv2D(8, (3, 3), activation='relu', padding='same')(xb)
xb = l.UpSampling2D((2, 2))(xb)
xb = l.Conv2D(16, (3, 3), activation='relu')(xb)
xb = l.UpSampling2D((2, 2))(xb)
decoded_b = l.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(xb)

autoencoder_a = keras.Model(input_img, decoded_a)
autoencoder_b = keras.Model(input_img, decoded_b)

encoder = keras.Model(input_img, encoded)

autoencoder_a.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder_b.compile(optimizer='adadelta', loss='binary_crossentropy')


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
autoencoder_a.fit_generator(
                    generator=training_generator_1,
                    validation_data=validation_generator_1,
                     epochs=5)

Epoch 1/5
187/187 [==============================] - 35s 186ms/step - loss: 0.6017 - val_loss: 0.5520
Epoch 2/5
163/187 [=========================>....] - ETA: 3s - loss: 0.5504

KeyboardInterrupt: 

In [162]:
train_xa = pics_a[:int(0.9*len(pics_a))]

In [163]:
train_xa = train_xa.reshape((len(train_xa), 100,100,1))

In [166]:
autoencoder_a.fit(train_xa, train_xa, epochs=5)

Epoch 1/5
12030/12030 [==============================] - 10s 798us/step - loss: -9718488953305814366420992.0000
Epoch 2/5
12030/12030 [==============================] - 10s 802us/step - loss: -12187318540289381276581888.0000
Epoch 3/5
 9984/12030 [=======================>......] - ETA: 1s - loss: -14454296433871135916949504.0000

KeyboardInterrupt: 

In [ ]:
autoencoder_a.fit

In [153]:
cv.resize(cv.cvtColor(cv.imread('/mnt/data/deepfake/datasets/faces/Actor_01/02-01-04-01-01-01-01.mp4_65.jpg'), cv.COLOR_BGR2GRAY), (DIM,DIM)).shape

(100, 100)

In [ ]:
ls /mnt/data/deepfake/datasets/faces/Ac

In [116]:
'02-01-07-01-02-02-02.mp4_38.jpg' in images_2

True

In [13]:
def get_pics(path, lim = 1000): 
    pics = []
    for i,pic in enumerate(os.listdir(path)):
        gray = cv.cvtColor(cv.imread(path + pic), cv.COLOR_BGR2GRAY)
        pics.append(gray)
        
        if i == lim:break
    return pics

def resize_pics(pics):
    resize = []
    for pic in pics:
        resize.append(cv.resize(pic, (DIM, DIM)))
    return np.array(resize)

In [14]:
pics_a = get_pics(dataset_1, lim = 9999999999999)

In [12]:
pics_a = resize_pics(pics_a)

NameError: name 'resize_pics' is not defined

In [167]:
del pics_a

In [168]:
del train_xa